In [1]:
# ---------------------------------------------------------------------
# Fixed Colab cell: Transfer-learning + Fine-tuning (EfficientNetB0)
# - Ensures AdamW optimizer
# - Removes EarlyStopping (training will run all epochs)
# - Saves/plots loss & accuracy, confusion matrix, ROC curve
# - Saves per-sample CSV predictions and final model
# ---------------------------------------------------------------------

# ---------------------------
# Imports
# ---------------------------
import os
import zipfile
import math
import time
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize
from collections import Counter

# ---------------------------
# Configuration / Settings
# ---------------------------
OPTIMIZER = "adamw"        # "adamw" | "adamax" | "sgd"
BATCH_SIZE = 32
IMG_SIZE = (224, 224)     # change if you want larger/smaller inputs
NUM_CLASSES = None        # leave None to auto-detect from train folder
EPOCHS_HEAD = 8           # Stage 1: train head
EPOCHS_FINETUNE = 42      # Stage 2: fine-tune
TOTAL_EPOCHS = EPOCHS_HEAD + EPOCHS_FINETUNE
SEED = 42

USE_MIXUP = False         # True to enable MixUp augmentation
USE_MIXUP_ALPHA = 0.2

# Paths (Google Drive)
DRIVE_MOUNT_POINT = "/content/drive"
ZIP_PATH = "/content/drive/MyDrive/d.zip"   # adjust to your zip path if needed
DATA_ROOT = "/content/dataset"
TRAIN_DIR = os.path.join(DATA_ROOT, "d", "train")
VAL_DIR   = os.path.join(DATA_ROOT, "d", "val")

OUT_DIR = f"/content/drive/MyDrive/fine_tune_results_{OPTIMIZER}"
CHECKPOINT_PATH = os.path.join(OUT_DIR, "best_model.keras")
FINAL_MODEL_PATH = os.path.join(OUT_DIR, "final_model.keras")
CSV_PRED = os.path.join(OUT_DIR, "val_predictions.csv")
METRICS_JSON = os.path.join(OUT_DIR, "training_metrics_summary.json")

# Optimizer / LR settings
LR_HEAD = 1e-3
LR_FINETUNE = 1e-5
WEIGHT_DECAY = 1e-6
CLIPNORM = None

# Mixed precision (optional)
USE_MIXED_PRECISION = False

# ---------------------------
# Mount Drive (Colab)
# ---------------------------
from google.colab import drive
drive.mount(DRIVE_MOUNT_POINT, force_remount=False)

# ---------------------------
# Repro / mixed precision
# ---------------------------
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

if USE_MIXED_PRECISION:
    try:
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
        print("Mixed precision enabled.")
    except Exception as e:
        print("Mixed precision not enabled:", e)

# ---------------------------
# Extract dataset if zipped (optional)
# ---------------------------
os.makedirs(DATA_ROOT, exist_ok=True)
if os.path.exists(ZIP_PATH) and not (os.path.exists(TRAIN_DIR) and os.path.exists(VAL_DIR)):
    print("Extracting dataset zip from Drive to", DATA_ROOT)
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(DATA_ROOT)
    print("Extraction complete.")
else:
    print("Dataset already present or zip not found. DATA_ROOT =", DATA_ROOT)

# ---------------------------
# Utility: class counts / weights
# ---------------------------
def class_counts_from_dir(dirpath):
    counts = {}
    if not os.path.exists(dirpath):
        return counts
    for cls in sorted(os.listdir(dirpath)):
        cls_path = os.path.join(dirpath, cls)
        if os.path.isdir(cls_path):
            counts[cls] = sum([1 for _ in tf.io.gfile.glob(os.path.join(cls_path, "*"))])
    return counts

train_counts = class_counts_from_dir(TRAIN_DIR)
val_counts = class_counts_from_dir(VAL_DIR)
print("Train class counts:", train_counts)
print("Val   class counts:", val_counts)

def make_class_weights(counts):
    if not counts:
        return None
    labels = sorted(counts.keys())
    freqs = np.array([counts[k] for k in labels], dtype=np.float32)
    total = freqs.sum()
    weights = (total / (len(labels) * freqs))
    return dict(zip(labels, weights))

class_weights_by_name = make_class_weights(train_counts)
print("Per-class weights by name:", class_weights_by_name)

# ---------------------------
# Build tf.data datasets via image_dataset_from_directory
# ---------------------------
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
NUM_CLASSES = NUM_CLASSES or len(class_names)
print("Detected classes (index -> name):", dict(enumerate(class_names)), "NUM_CLASSES=", NUM_CLASSES)

# Convert class_weights_by_name -> class_weights (index: weight)
class_weights = None
if class_weights_by_name:
    class_weights = {i: float(class_weights_by_name[name]) for i, name in enumerate(class_names) if name in class_weights_by_name}
    print("Class weights (index):", class_weights)

# ---------------------------
# Preprocessing & augmentation
# ---------------------------
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.12),
    tf.keras.layers.RandomZoom(0.12),
    tf.keras.layers.RandomContrast(0.08),
], name="data_augmentation")

# Mixup util
def sample_beta_distribution(size, concentration=USE_MIXUP_ALPHA):
    return np.random.beta(concentration, concentration, size)

def mixup(batch_x, batch_y, alpha=USE_MIXUP_ALPHA):
    lam = sample_beta_distribution(batch_x.shape[0]).astype('float32')
    lam_x = lam.reshape(-1, 1, 1, 1)
    lam_y = lam.reshape(-1, 1)
    index = np.random.permutation(batch_x.shape[0])
    mixed_x = batch_x * lam_x + batch_x[index] * (1 - lam_x)
    y1 = tf.one_hot(batch_y, NUM_CLASSES)
    y2 = tf.one_hot(batch_y[index], NUM_CLASSES)
    mixed_y = y1 * lam_y + y2 * (1 - lam_y)
    return mixed_x, mixed_y

def prepare_train(ds, augment=True, mixup_prob=0.0):
    def _map(x, y):
        x = tf.cast(x, tf.float32)
        x = effnet_preprocess(x)
        if augment:
            x = data_augmentation(x)
        return x, y
    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    ds = ds.cache().prefetch(AUTOTUNE)
    if USE_MIXUP and mixup_prob > 0.0:
        def _mixup(x, y):
            x_m, y_m = mixup(x, y, alpha=mixup_prob)
            return x_m.astype('float32'), y_m.astype('float32')
        def _mixup_tf(x, y):
            x_m, y_m = tf.numpy_function(_mixup, [x, y], [tf.float32, tf.float32])
            x_m.set_shape([None, IMG_SIZE[0], IMG_SIZE[1], 3])
            y_m.set_shape([None, NUM_CLASSES])
            return x_m, y_m
        ds = ds.map(_mixup_tf, num_parallel_calls=AUTOTUNE)
    else:
        def _to_onehot(x, y):
            return x, tf.one_hot(y, NUM_CLASSES)
        ds = ds.map(_to_onehot, num_parallel_calls=AUTOTUNE)
    return ds

def prepare_val(ds):
    def _map(x,y):
        x = tf.cast(x, tf.float32)
        x = effnet_preprocess(x)
        return x, y
    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    ds = ds.cache().prefetch(AUTOTUNE)
    return ds

train_ds_prepared = prepare_train(train_ds, augment=True, mixup_prob=USE_MIXUP_ALPHA if USE_MIXUP else 0.0)
val_ds_prepared = prepare_val(val_ds)

# ---------------------------
# Model: EfficientNetB0 base + head
# ---------------------------
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

def build_model(input_shape=(*IMG_SIZE, 3), n_classes=NUM_CLASSES, dropout_rate=0.4):
    base = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')
    x = base.output
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(n_classes, activation='softmax', dtype='float32')(x)
    model = models.Model(inputs=base.input, outputs=outputs)
    return model, base

model, base_model = build_model()
model.summary()

# ---------------------------
# Optimizer selection (AdamW preferred)
# ---------------------------
def get_optimizer(name="adamw", lr=LR_HEAD, weight_decay=WEIGHT_DECAY, clipnorm=CLIPNORM):
    name = name.lower().strip()
    if name == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True, clipnorm=clipnorm)
    if name == "adamax":
        return tf.keras.optimizers.Adamax(learning_rate=lr, clipnorm=clipnorm)
    if name == "adamw":
        try:
            return tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=weight_decay, clipnorm=clipnorm)
        except Exception:
            try:
                return tf.keras.optimizers.experimental.AdamW(learning_rate=lr, weight_decay=weight_decay, clipnorm=clipnorm)
            except Exception:
                print("WARNING: AdamW not available — falling back to Adamax.")
                return tf.keras.optimizers.Adamax(learning_rate=lr, clipnorm=clipnorm)
    raise ValueError("Unsupported optimizer: " + name)

# ---------------------------
# Callbacks
# ---------------------------
os.makedirs(OUT_DIR, exist_ok=True)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    CHECKPOINT_PATH, monitor='val_loss', save_best_only=True, save_weights_only=False, verbose=1
)
reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-7, verbose=1)
# DO NOT use EarlyStopping to ensure training runs full epochs
tensorboard_logdir = os.path.join(OUT_DIR, "tb_logs")
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_logdir, histogram_freq=1)

# ---------------------------
# Stage 1: Train head only (base frozen)
# ---------------------------
base_model.trainable = False
for layer in base_model.layers:
    layer.trainable = False

optimizer_stage1 = get_optimizer(OPTIMIZER, lr=LR_HEAD, weight_decay=WEIGHT_DECAY)
model.compile(optimizer=optimizer_stage1, loss='categorical_crossentropy', metrics=['accuracy'])

print("\n=== Stage 1: Training head (base frozen) ===")
history_head = model.fit(
    train_ds_prepared,
    validation_data=val_ds_prepared.map(lambda x,y:(x, tf.one_hot(y, NUM_CLASSES))),
    epochs=EPOCHS_HEAD,
    callbacks=[checkpoint_cb, reduce_lr_cb, tensorboard_cb],
    class_weight=class_weights,
    verbose=1
)

# ---------------------------
# Stage 2: Fine-tune (unfreeze base; small LR)
# ---------------------------
# Unfreeze base
base_model.trainable = True
# Optionally freeze batchnorm layers for stability
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

optimizer_stage2 = get_optimizer(OPTIMIZER, lr=LR_FINETUNE, weight_decay=WEIGHT_DECAY)
model.compile(optimizer=optimizer_stage2, loss='categorical_crossentropy', metrics=['accuracy'])

print("\n=== Stage 2: Fine-tuning (base unfrozen) ===")
initial_epoch_for_finetune = EPOCHS_HEAD
history_finetune = model.fit(
    train_ds_prepared,
    validation_data=val_ds_prepared.map(lambda x,y:(x, tf.one_hot(y, NUM_CLASSES))),
    epochs=TOTAL_EPOCHS,
    initial_epoch=initial_epoch_for_finetune,
    callbacks=[checkpoint_cb, reduce_lr_cb, tensorboard_cb],
    class_weight=class_weights,
    verbose=1
)

# ---------------------------
# Combine history objects for plotting
# ---------------------------
def combine_histories(h1, h2):
    # h1 and h2 are keras History objects (for stage1 and stage2 respectively)
    out = {}
    for k in h1.history.keys():
        out[k] = h1.history[k] + h2.history.get(k, [])
    # include any keys only in h2
    for k in h2.history.keys():
        if k not in out:
            out[k] = h2.history[k]
    return out

combined = combine_histories(history_head, history_finetune)

# ---------------------------
# Plot training curves (loss & accuracy)
# ---------------------------
def plot_and_save_training_curves(history_dict, out_dir):
    epochs = range(1, len(history_dict.get('loss', [])) + 1)
    # Accuracy
    if 'accuracy' in history_dict:
        plt.figure(figsize=(8,6))
        plt.plot(epochs, history_dict['accuracy'], label='train_acc')
        if 'val_accuracy' in history_dict:
            plt.plot(epochs, history_dict['val_accuracy'], label='val_acc')
        plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Training & Validation Accuracy')
        plt.legend(); plt.grid(True)
        p = os.path.join(out_dir, "accuracy_curve.png")
        plt.savefig(p); plt.close()
        print("Saved accuracy plot to:", p)
    # Loss
    if 'loss' in history_dict:
        plt.figure(figsize=(8,6))
        plt.plot(epochs, history_dict['loss'], label='train_loss')
        if 'val_loss' in history_dict:
            plt.plot(epochs, history_dict['val_loss'], label='val_loss')
        plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Training & Validation Loss')
        plt.legend(); plt.grid(True)
        p = os.path.join(out_dir, "loss_curve.png")
        plt.savefig(p); plt.close()
        print("Saved loss plot to:", p)

plot_and_save_training_curves(combined, OUT_DIR)

# ---------------------------
# Save final model
# ---------------------------
try:
    model.save(FINAL_MODEL_PATH)
    print("Saved final model to:", FINAL_MODEL_PATH)
except Exception as e:
    print("Saving .keras failed, attempting .h5 fallback:", e)
    try:
        fallback = FINAL_MODEL_PATH.replace(".keras", ".h5")
        model.save(fallback)
        print("Saved fallback model to:", fallback)
    except Exception as e2:
        print("Final save also failed:", e2)

# ---------------------------
# Load best checkpoint (if available)
# ---------------------------
best_model_path = CHECKPOINT_PATH if os.path.exists(CHECKPOINT_PATH) else FINAL_MODEL_PATH
print("Attempting to load best model from:", best_model_path)
try:
    model = tf.keras.models.load_model(best_model_path)
    print("Loaded model:", best_model_path)
except Exception as e:
    print("Could not load checkpoint; continuing with in-memory model. Error:", e)

# ---------------------------
# Evaluate on validation & predictions CSV
# ---------------------------
# Reconstruct validation file list in same deterministic order used by image_dataset_from_directory
val_filepaths = []
for class_idx, cls_name in enumerate(class_names):
    cls_dir = os.path.join(VAL_DIR, cls_name)
    if not os.path.isdir(cls_dir):
        continue
    files = sorted([f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))])
    for fname in files:
        val_filepaths.append(os.path.join(cls_name, fname))

n_val = len(val_filepaths)
print(f"Found {n_val} validation files (reconstructed).")

# Build a prediction dataset with shuffle=False to preserve order
val_pred_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

# Preprocess exactly as training (EfficientNet preprocess)
val_pred_ds_pp = val_pred_ds.map(lambda x,y: (effnet_preprocess(tf.cast(x, tf.float32)), y)).prefetch(AUTOTUNE)

# Predict
pred_probs = model.predict(val_pred_ds_pp, verbose=1)

# Extract true labels from the unshuffled val_pred_ds (as numpy array)
y_true = np.concatenate([y.numpy() for x, y in val_pred_ds], axis=0)
y_pred = np.argmax(pred_probs, axis=1)

# Sanity checks
if pred_probs.shape[0] != len(y_true):
    print("WARNING: number of predictions does not match number of true labels.")
    print("pred_probs rows:", pred_probs.shape[0], "y_true len:", len(y_true), "reconstructed filenames:", len(val_filepaths))

# Build mapping index->label
index_to_label = {i: name for i, name in enumerate(class_names)}
target_names = [index_to_label[i] for i in sorted(index_to_label.keys())]

# Accuracy & confusion matrix
acc = accuracy_score(y_true, y_pred) if len(y_true) > 0 else 0.0
print(f"\nValidation accuracy: {acc*100:.2f}%")

cm = confusion_matrix(y_true, y_pred, labels=sorted(index_to_label.keys()))
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted"); plt.ylabel("Actual"); plt.title("Confusion Matrix")
cm_path = os.path.join(OUT_DIR, "confusion_matrix.png")
plt.savefig(cm_path)
plt.close()
print("Saved confusion matrix to:", cm_path)

print("\nClassification report:\n")
print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))

# ROC per-class (one-vs-rest)
roc_path = os.path.join(OUT_DIR, "roc_curve.png")
try:
    y_true_bin = label_binarize(y_true, classes=list(range(NUM_CLASSES)))
    fpr, tpr, roc_auc = {}, {}, {}
    present_classes = []
    for i in range(NUM_CLASSES):
        if np.sum(y_true_bin[:, i]) == 0:
            print(f"Skipping ROC for class {i} ({index_to_label.get(i,'?')}) — no positives in y_true")
            continue
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        present_classes.append(i)

    plt.figure(figsize=(9,8))
    for i in present_classes:
        plt.plot(fpr[i], tpr[i], label=f"{index_to_label.get(i, i)} (AUC={roc_auc[i]:.3f})")
    plt.plot([0,1],[0,1],'k--', label='Chance')
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate"); plt.title("ROC Curves (one-vs-rest)")
    plt.legend(loc='lower right'); plt.grid(True)
    plt.savefig(roc_path)
    plt.close()
    print("Saved ROC plot to:", roc_path)
except Exception as e:
    print("ROC plotting skipped due to:", e)

# For binary classification, also compute single AUC using positive class probs (index 1)
auc_score = None
try:
    if NUM_CLASSES == 2 and pred_probs.shape[1] >= 2 and len(y_true) > 0:
        auc_score = roc_auc_score(y_true, pred_probs[:, 1])
        print(f"Binary ROC AUC (class '{index_to_label.get(1,1)}' as positive): {auc_score:.4f}")
except Exception as e:
    print("Binary AUC computation failed:", e)

# Save CSV of predictions (use reconstructed filenames if available)
rows = []
min_len = min(len(val_filepaths), len(y_true), pred_probs.shape[0])
if min_len < len(y_true):
    print("NOTE: filename count differs from true/pred count — using min_len =", min_len)

for i in range(min_len):
    filename = val_filepaths[i] if i < len(val_filepaths) else f"unknown_{i}"
    rows.append({
        "filename": filename,
        "true_label": index_to_label[int(y_true[i])],
        "pred_label": index_to_label[int(y_pred[i])],
        **{f"prob_{index_to_label[j]}": float(pred_probs[i, j]) for j in range(pred_probs.shape[1])}
    })

# If there are leftover samples (rare), append them safely
if len(y_true) > min_len:
    for i in range(min_len, len(y_true)):
        rows.append({
            "filename": f"unknown_{i}",
            "true_label": index_to_label[int(y_true[i])],
            "pred_label": index_to_label[int(y_pred[i])],
            **{f"prob_{index_to_label[j]}": float(pred_probs[i, j]) for j in range(pred_probs.shape[1])}
        })

df = pd.DataFrame(rows)
df.to_csv(CSV_PRED, index=False)
print("Saved per-sample predictions to:", CSV_PRED)

# ---------------------------
# Save summary metrics & training info
# ---------------------------
metrics_summary = {
    "num_classes": int(NUM_CLASSES),
    "class_names": class_names,
    "train_counts": train_counts,
    "val_counts": val_counts,
    "final_validation_accuracy": float(acc),
    "binary_auc_positive_class_1": float(auc_score) if auc_score is not None else None,
    "epochs_head": int(EPOCHS_HEAD),
    "epochs_finetune": int(EPOCHS_FINETUNE),
    "total_epochs": int(TOTAL_EPOCHS),
    "optimizer": OPTIMIZER,
    "learning_rate_head": LR_HEAD,
    "learning_rate_finetune": LR_FINETUNE,
    "class_weights": class_weights,
    "artifacts": {
        "checkpoint": best_model_path,
        "final_model": FINAL_MODEL_PATH,
        "predictions_csv": CSV_PRED,
        "confusion_matrix": cm_path,
        "roc_curve": roc_path,
        "accuracy_curve": os.path.join(OUT_DIR, "accuracy_curve.png"),
        "loss_curve": os.path.join(OUT_DIR, "loss_curve.png")
    }
}

with open(METRICS_JSON, "w") as f:
    json.dump(metrics_summary, f, indent=2)
print("Saved training summary JSON to:", METRICS_JSON)

print("\nEvaluation complete. Artifacts stored under:", OUT_DIR)


Mounted at /content/drive
Extracting dataset zip from Drive to /content/dataset
Extraction complete.
Train class counts: {'benign': 1440, 'malignant': 1197}
Val   class counts: {'benign': 360, 'malignant': 300}
Per-class weights by name: {'benign': np.float32(0.915625), 'malignant': np.float32(1.1015037)}
Found 2637 files belonging to 2 classes.
Found 660 files belonging to 2 classes.
Detected classes (index -> name): {0: 'benign', 1: 'malignant'} NUM_CLASSES= 2
Class weights (index): {0: 0.9156249761581421, 1: 1.1015037298202515}
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,052,133 (15.46 MB)

 Trainable params: 4,010,110 (15.30 MB)

 Non-trainable params: 42,023 (164.16 KB)


=== Stage 1: Training head (base frozen) ===
Epoch 1/8
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 518ms/step - accuracy: 0.6823 - loss: 0.5977
Epoch 1: val_loss improved from inf to 0.42634, saving model to /content/drive/MyDrive/fine_tune_results_adamw/best_model.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 101s 804ms/step - accuracy: 0.6831 - loss: 0.5967 - val_accuracy: 0.8197 - val_loss: 0.4263 - learning_rate: 0.0010
Epoch 2/8
82/83 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.8154 - loss: 0.4147
Epoch 2: val_loss improved from 0.42634 to 0.38995, saving model to /content/drive/MyDrive/fine_tune_results_adamw/best_model.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 60s 101ms/step - accuracy: 0.8154 - loss: 0.4145 - val_accuracy: 0.8303 - val_loss: 0.3900 - learning_rate: 0.0010
Epoch 3/8
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8185 - loss: 0.3929
Epoch 3: val_loss improved from 0.38995 to 0.37289, saving model to /content/drive/MyDrive/fine_tune_results_adamw/best_model.keras
83/83 ━━━━━━━━━━━━━━━━━━